# Binäre Klassifikation

Der typische Ablauf eines binären Klassifikationsproblems in Python kann in folgenden Schritten zusammengefasst werden:

* **Datensatz laden**: Der erste Schritt besteht darin, den Datensatz zu laden und in eine Datenstruktur wie z.B. ein Pandas DataFrame zu laden.
* **Datenvorverarbeitung**: In diesem Schritt werden die Daten vorbereitet und aufbereitet, um sie für die Analyse und das Modellieren vorzubereiten. Dazu gehören Schritte wie das Entfernen von fehlenden Werten, das Skalieren von Features und das Codieren von kategorialen Variablen.
* **Feature-Engineering**: Hier werden neue Features aus den vorhandenen Daten erstellt, um das Modell zu verbessern. Beispiele hierfür sind die Extraktion von Zeitmerkmalen aus Datumsangaben oder die Kombination von Features.
* **Datenaufteilung**: In diesem Schritt wird der Datensatz in Trainings- und Testdaten aufgeteilt, um das Modell zu trainieren und zu testen. Eine typische Aufteilung ist z.B. 70% Training und 30% Test.
* **Modellierung**: Nun wird das Klassifikationsmodell erstellt und trainiert. Hierfür gibt es verschiedene Algorithmen wie z.B. Logistic Regression, Decision Trees, Random Forests oder Support Vector Machines.
* **Modellbewertung**: Nachdem das Modell trainiert wurde, wird es auf den Testdaten evaluiert, um die Leistung des Modells zu bewerten. Dazu werden Metriken wie Accuracy, Precision, Recall und F1-Score verwendet.
* **Hyperparameter-Optimierung**: In diesem Schritt werden die Hyperparameter des Modells optimiert, um die Leistung weiter zu verbessern. Hierfür gibt es verschiedene Techniken wie z.B. Grid Search oder Random Search.
* **Vorhersagen treffen**: Nachdem das Modell optimiert wurde, wird es verwendet, um Vorhersagen für neue Daten zu treffen.
* **Modell-Deployment**: Schließlich wird das Modell in eine produktionsreife Umgebung implementiert, um Echtzeitvorhersagen zu treffen.


# Daten laden

In [ ]:
import pandas as pd
automaten = pd.read_csv("https://www.offenedaten-koeln.de/sites/default/files/psa_offene_daten_2023.csv", encoding='unicode_escape', sep=";")

strafen = pd.read_csv("https://www.offenedaten-koeln.de/sites/default/files/Bussgeld_2021.csv", encoding='unicode_escape', sep=";")

tatbestände = pd.read_csv("https://raw.githubusercontent.com/jomo/bkat-owi/master/data.csv")

# Datensatz vorbereiten

In [ ]:
tatbestände_aktuell = tatbestände.drop_duplicates("Tatbestandsnummer",keep="last").set_index("Tatbestandsnummer")
joined = strafen.join(tatbestände_aktuell[["Euro","Tatbestand_Druckdatei","Klassifizierung"]], on="tatbestand1")

Außerdem brauchen wir eine Zielvariable. Der einfachheithalber versuchen wir einfach nur vorherzusagen, ob es sich um Parken ohne Parkticket handelt oder nicht. Prinzipiell könnten wir aber alles mögliche versuchen vorherzusagen, solange diese Informationen verfügbar sind.

In [ ]:
joined["no_parking_ticket"] = joined.Tatbestand_Druckdatei == "Sie parkten im Bereich eines Parkscheinautomaten ohne gültigen\nParkschein."

In [ ]:
joined.no_parking_ticket.value_counts()

# Feature-Engineering

Die meisten Maschine-Learning-Verfahren können nur mit nummerischen Daten umgehen. Spalten mit nicht nummerischen Informationen müssen entsprechend als Kategorien enkodiert werden:

In [ ]:
from sklearn.preprocessing import LabelEncoder  
categorical_columns = ["fahrzeugart", "fabrikat"]
for col in categorical_columns:  
    le = LabelEncoder()  
    joined.loc[:,col] = le.fit_transform(joined[col].astype(str))
joined[categorical_columns] = joined[categorical_columns].astype('category')  

Außerdem können wir zusätzliche Informationen aus anderen Informationen ableiten

In [ ]:
time = pd.to_datetime(joined.datum_von)

In [ ]:
datetime_features = pd.DataFrame({
    "dayofyear": time.dt.dayofyear / 354,
    "wochentag": time.dt.weekday / 7
})
parkverstoesse = joined.join(datetime_features)

# Datenaufteilung

In [ ]:
from sklearn.model_selection import train_test_split
zielvariable="no_parking_ticket"
features = ["fahrzeugart", "fabrikat","dayofyear","wochentag"]

# Ungültige Spalten Entfernen
parkverstoesse.dropna(subset=[zielvariable], inplace=True)

X = parkverstoesse[features]
y = parkverstoesse[zielvariable]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Modellierung

In [ ]:
from lightgbm import LGBMClassifier
gbc = LGBMClassifier()
gbc.fit(X_train, y_train)

# Modellbewertung

In [ ]:
y_pred = gbc.predict(X_test)  

In [ ]:
from sklearn.metrics import accuracy_score, classification_report 

# Evaluating  
accuracy = accuracy_score(y_test, y_pred)  
print("Accuracy:", accuracy)  
print("Classification Report:\n", classification_report(y_test, y_pred))  

In [ ]:
pd.Series(dict(zip(X_train.columns, gbc.feature_importances_))).plot.barh()

# Modellverbesserung

Wir haben ein erstes Modell :-) Bevor wir unser Modell aber deployen und in der echten Welt nutzen, sollten wir es noch etwas analysieren. Wie könnte man das Modell noch verbessern? Worauf muss man achten?

 * Hyperparameter-Optimierung: Man könnte unterschiedliche Parameter für unser Modell testen.
 * ...

# Bonus: Unbalancierte Daten

Ein typisches Problem ist, dass die Zielvariable nicht gleichmäßig verteilt ist. Insbesondere bei stark unterschiedlich ausgeprägten Datenklassen muss man hiermit umgehen. Eine Möglichkeit ist, die häufiger vertretenen Klassen herab zu samplen

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train, y_train)

In [ ]:
from lightgbm import LGBMClassifier
gbc = LGBMClassifier()
gbc.fit(X_rus, y_rus)

In [ ]:
y_pred = gbc.predict(X_test)  

In [ ]:
# Evaluating  
accuracy = accuracy_score(y_test, y_pred)  
print("Accuracy:", accuracy)  
print("Classification Report:\n", classification_report(y_test, y_pred))  

In [ ]:
pd.Series(dict(zip(X_train.columns, gbc.feature_importances_))).plot.barh()